Load in our labeled data

In [212]:
import pandas as pd
from dateutil import parser

dataset = 'ML_Data/dj_classified_pos.xlsx'
output_dataset_name = 'ML_Data/DJI_pos.csv'
price_data = 'data/price/DJI.xlsx'
dates = 'data/dates.csv'

df = pd.read_excel(dataset)
df.columns = ['date', 'comment', 'label', 'compound_score']
df['date'] = pd.to_datetime(df['date'], dayfirst=True)

In [213]:
del df['comment']
del df['compound_score']

In [214]:
df.head()

,date,label
0,2019-05-03,0
1,2019-05-03,1
2,2019-05-03,0
3,2019-05-03,0
4,2019-05-03,0


Loop through all comments and create a new DataFrame that hosts the number of positives and negatives for each day.

In [215]:
rows = []
prices = []
grouped = df.groupby('date')
for date, group in grouped:
    positives = len(group[group['label'] == 1])
    negatives = len(group[group['label'] == 0])
        
    new_row = {
        'date': date,
        'positives': positives,
        'negatives': negatives,
        'ratio': positives / (positives + negatives)
    }
    
    rows.append(new_row)

In [216]:
new_df = pd.DataFrame(rows)

In [217]:
new_df.tail()

,date,negatives,positives,ratio
329,2019-04-29,4,3,0.428571
330,2019-04-30,5,3,0.375000
331,2019-05-01,13,10,0.434783
332,2019-05-02,15,8,0.347826
333,2019-05-03,10,5,0.333333


# Join with dates, so we make sure we have all dates

In [218]:
df_dates = pd.read_csv(dates)
df_dates['date'] = pd.to_datetime(df_dates['date'])
df_dates.tail(10)

,date
478,2018-01-10
479,2018-01-09
480,2018-01-08
481,2018-01-07
482,2018-01-06
483,2018-01-05
484,2018-01-04
485,2018-01-03
486,2018-01-02
487,2018-01-01


In [219]:
new_df = pd.merge(df_dates, new_df, on='date', how='outer')

In [220]:
new_df.fillna(0, inplace=True)

In [221]:
new_df.head(10)

,date,negatives,positives,ratio
0,2019-05-03,10.0,5.0,0.333333
1,2019-05-02,15.0,8.0,0.347826
2,2019-05-01,13.0,10.0,0.434783
3,2019-04-30,5.0,3.0,0.375000
4,2019-04-29,4.0,3.0,0.428571
5,2019-04-28,0.0,0.0,0.000000
6,2019-04-27,0.0,0.0,0.000000
7,2019-04-26,13.0,10.0,0.434783
8,2019-04-25,18.0,7.0,0.280000
9,2019-04-24,7.0,5.0,0.416667


Loop through new DataFrame and add 2 lags to each day. First we reverse order of DataFrame.

In [222]:
new_df = new_df.iloc[::-1]

lags = range(1, 8)

In [223]:
new_df = new_df.assign(**{
    '{} (t-{})'.format(col, t): new_df[col].shift(t)
    for t in lags
    for col in new_df
})

In [224]:
del new_df['date (t-1)']
del new_df['date (t-2)']
del new_df['date (t-3)']
del new_df['date (t-4)']
del new_df['date (t-5)']
del new_df['date (t-6)']
del new_df['date (t-7)']

In [225]:
new_df.fillna(0, inplace=True)
new_df.head(10)

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,ratio (t-4),negatives (t-5),positives (t-5),ratio (t-5),negatives (t-6),positives (t-6),ratio (t-6),negatives (t-7),positives (t-7),ratio (t-7)
487,2018-01-01,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
486,2018-01-02,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
485,2018-01-03,7.0,12.0,0.631579,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
484,2018-01-04,22.0,32.0,0.592593,7.0,12.0,0.631579,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
483,2018-01-05,59.0,35.0,0.372340,22.0,32.0,0.592593,7.0,12.0,0.631579,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
482,2018-01-06,0.0,0.0,0.000000,59.0,35.0,0.372340,22.0,32.0,0.592593,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
481,2018-01-07,0.0,0.0,0.000000,0.0,0.0,0.000000,59.0,35.0,0.372340,...,0.631579,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
480,2018-01-08,15.0,16.0,0.516129,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.592593,7.0,12.0,0.631579,0.0,0.0,0.000000,0.0,0.0,0.000000
479,2018-01-09,29.0,29.0,0.500000,15.0,16.0,0.516129,0.0,0.0,0.000000,...,0.372340,22.0,32.0,0.592593,7.0,12.0,0.631579,0.0,0.0,0.000000
478,2018-01-10,35.0,23.0,0.396552,29.0,29.0,0.500000,15.0,16.0,0.516129,...,0.000000,59.0,35.0,0.372340,22.0,32.0,0.592593,7.0,12.0,0.631579


In [226]:
new_df = new_df.sort_values(['date'],ascending=[False])

In [227]:
new_df.head(10)

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,ratio (t-4),negatives (t-5),positives (t-5),ratio (t-5),negatives (t-6),positives (t-6),ratio (t-6),negatives (t-7),positives (t-7),ratio (t-7)
0,2019-05-03,10.0,5.0,0.333333,15.0,8.0,0.347826,13.0,10.0,0.434783,...,0.428571,0.0,0.0,0.000000,0.0,0.0,0.000000,13.0,10.0,0.434783
1,2019-05-02,15.0,8.0,0.347826,13.0,10.0,0.434783,5.0,3.0,0.375000,...,0.000000,0.0,0.0,0.000000,13.0,10.0,0.434783,18.0,7.0,0.280000
2,2019-05-01,13.0,10.0,0.434783,5.0,3.0,0.375000,4.0,3.0,0.428571,...,0.000000,13.0,10.0,0.434783,18.0,7.0,0.280000,7.0,5.0,0.416667
3,2019-04-30,5.0,3.0,0.375000,4.0,3.0,0.428571,0.0,0.0,0.000000,...,0.434783,18.0,7.0,0.280000,7.0,5.0,0.416667,16.0,5.0,0.238095
4,2019-04-29,4.0,3.0,0.428571,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.280000,7.0,5.0,0.416667,16.0,5.0,0.238095,7.0,4.0,0.363636
5,2019-04-28,0.0,0.0,0.000000,0.0,0.0,0.000000,13.0,10.0,0.434783,...,0.416667,16.0,5.0,0.238095,7.0,4.0,0.363636,0.0,0.0,0.000000
6,2019-04-27,0.0,0.0,0.000000,13.0,10.0,0.434783,18.0,7.0,0.280000,...,0.238095,7.0,4.0,0.363636,0.0,0.0,0.000000,0.0,0.0,0.000000
7,2019-04-26,13.0,10.0,0.434783,18.0,7.0,0.280000,7.0,5.0,0.416667,...,0.363636,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
8,2019-04-25,18.0,7.0,0.280000,7.0,5.0,0.416667,16.0,5.0,0.238095,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,6.0,16.0,0.727273
9,2019-04-24,7.0,5.0,0.416667,16.0,5.0,0.238095,7.0,4.0,0.363636,...,0.000000,0.0,0.0,0.000000,6.0,16.0,0.727273,9.0,4.0,0.307692


# Merge with price data

In [228]:
df_price = pd.read_excel(price_data, skiprows=[0])
df_price['date'] = pd.to_datetime(df_price['Date'])

new_df = pd.merge(new_df, df_price, how='inner', on='date')

In [229]:
new_df.head(10)

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,ratio (t-6),negatives (t-7),positives (t-7),ratio (t-7),Date,High,Low,Upper Bound,Lower Bound,Close
0,2019-05-03,10.0,5.0,0.333333,15.0,8.0,0.347826,13.0,10.0,0.434783,...,0.000000,13.0,10.0,0.434783,5/3/19,26534.960938,26370.089844,33168.701172,19777.567383,26504.949219
1,2019-05-02,15.0,8.0,0.347826,13.0,10.0,0.434783,5.0,3.0,0.375000,...,0.434783,18.0,7.0,0.280000,5/2/19,26454.689453,26180.359375,33068.361816,19635.269531,26307.789063
2,2019-05-01,13.0,10.0,0.434783,5.0,3.0,0.375000,4.0,3.0,0.428571,...,0.280000,7.0,5.0,0.416667,5/1/19,26689.390625,26426.380859,33361.738281,19819.785644,26430.140625
3,2019-04-30,5.0,3.0,0.375000,4.0,3.0,0.428571,0.0,0.0,0.000000,...,0.416667,16.0,5.0,0.238095,4/30/19,26614.039063,26419.470703,33267.548829,19814.603027,26592.910156
4,2019-04-29,4.0,3.0,0.428571,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.238095,7.0,4.0,0.363636,4/29/19,26602.539063,26520.750000,33253.173829,19890.562500,26554.390625
5,2019-04-26,13.0,10.0,0.434783,18.0,7.0,0.280000,7.0,5.0,0.416667,...,0.000000,0.0,0.0,0.000000,4/26/19,26543.560547,26392.550781,33179.450684,19794.413086,26543.330078
6,2019-04-25,18.0,7.0,0.280000,7.0,5.0,0.416667,16.0,5.0,0.238095,...,0.000000,6.0,16.0,0.727273,4/25/19,26536.480469,26310.279297,33170.600586,19732.709473,26462.080078
7,2019-04-24,7.0,5.0,0.416667,16.0,5.0,0.238095,7.0,4.0,0.363636,...,0.727273,9.0,4.0,0.307692,4/24/19,26680.580078,26582.859375,33350.725097,19937.144531,26597.050781
8,2019-04-23,16.0,5.0,0.238095,7.0,4.0,0.363636,0.0,0.0,0.000000,...,0.307692,15.0,6.0,0.285714,4/23/19,26695.960938,26503.560547,33369.951172,19877.670410,26656.390625
9,2019-04-22,7.0,4.0,0.363636,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.285714,11.0,5.0,0.312500,4/22/19,26553.050781,26458.609375,33191.313476,19843.957031,26511.050781


In [208]:
new_df['label'] = new_df['Close'] > new_df['Close'].shift(-1)
new_df['label'] = new_df['label'].astype(int)

In [209]:
new_df.fillna(0, inplace=True)
new_df = new_df.drop(['High', 'Low', 'Upper Bound', 'Lower Bound', 'Close', 'Date'], axis=1)

In [210]:
new_df.head(10)

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,negatives (t-5),positives (t-5),ratio (t-5),negatives (t-6),positives (t-6),ratio (t-6),negatives (t-7),positives (t-7),ratio (t-7),label
0,2019-05-03,19.0,20.0,0.512821,26.0,23.0,0.469388,32.0,15.0,0.319149,...,0.0,0.0,0.000000,0.0,0.0,0.000000,65.0,37.0,0.362745,1
1,2019-05-02,26.0,23.0,0.469388,32.0,15.0,0.319149,15.0,12.0,0.444444,...,0.0,0.0,0.000000,65.0,37.0,0.362745,87.0,46.0,0.345865,1
2,2019-05-01,32.0,15.0,0.319149,15.0,12.0,0.444444,22.0,11.0,0.333333,...,65.0,37.0,0.362745,87.0,46.0,0.345865,59.0,42.0,0.415842,0
3,2019-04-30,15.0,12.0,0.444444,22.0,11.0,0.333333,0.0,0.0,0.000000,...,87.0,46.0,0.345865,59.0,42.0,0.415842,16.0,16.0,0.500000,0
4,2019-04-29,22.0,11.0,0.333333,0.0,0.0,0.000000,0.0,0.0,0.000000,...,59.0,42.0,0.415842,16.0,16.0,0.500000,22.0,13.0,0.371429,1
5,2019-04-26,65.0,37.0,0.362745,87.0,46.0,0.345865,59.0,42.0,0.415842,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0
6,2019-04-25,87.0,46.0,0.345865,59.0,42.0,0.415842,16.0,16.0,0.500000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,5.0,10.0,0.666667,0
7,2019-04-24,59.0,42.0,0.415842,16.0,16.0,0.500000,22.0,13.0,0.371429,...,0.0,0.0,0.000000,5.0,10.0,0.666667,3.0,4.0,0.571429,0
8,2019-04-23,16.0,16.0,0.500000,22.0,13.0,0.371429,0.0,0.0,0.000000,...,5.0,10.0,0.666667,3.0,4.0,0.571429,11.0,9.0,0.450000,1
9,2019-04-22,22.0,13.0,0.371429,0.0,0.0,0.000000,0.0,0.0,0.000000,...,3.0,4.0,0.571429,11.0,9.0,0.450000,17.0,6.0,0.260870,0


In [211]:
new_df.to_csv(output_dataset_name)